In [1]:
from datetime import datetime
import concurrent.futures
import numpy as np
from rdflib import URIRef
import itertools
from datetime import datetime
from collections import defaultdict, Counter, namedtuple
import gzip
import scipy
import random
import json
import scipy.sparse
import csv
import rdflib

INFO:rdflib:RDFLib Version: 4.2.1


In [2]:
class Encoder:
    def __init__(self):
        self.data2num = {}
        self.num2data = []
    
    def add(self, value):
        if value not in self.data2num:
            v = len(self.num2data)
            self.data2num[value] = v
            self.num2data.append(value)
            return v
        else:
            return self(value)
    
    def __call__(self, value):
        return self.data2num[value]
    
    def __getitem__(self, num):
        return self.num2data[num]
    
    def __len__(self):
        return len(self.num2data)

In [3]:
graph = []

soenc = Encoder()
penc = Encoder()

with gzip.open('DBpedia201610/mappingbased_objects_en.ttl.gz', 'rt') as f:
    for n,line in enumerate(f):
        if line[0] == '#':
            continue
#         if ignored_triples == 100:
#             break
        if (n+1)%500000 == 0:
            print("Line", n+1)
        line = line.strip()
        line = line[:-1]
        line = line.split()
        assert len(line) == 3
        if line[1].startswith("<http://dbpedia.org/ontology/"):            
            s = soenc.add(line[0])
            p = penc.add(line[1])
            o = soenc.add(line[2])
            graph.append((s, p, o))            
            
len(graph)

Line 500000
Line 1000000
Line 1500000
Line 2000000
Line 2500000
Line 3000000
Line 3500000
Line 4000000
Line 4500000
Line 5000000
Line 5500000
Line 6000000
Line 6500000
Line 7000000
Line 7500000
Line 8000000
Line 8500000
Line 9000000
Line 9500000
Line 10000000
Line 10500000
Line 11000000
Line 11500000
Line 12000000
Line 12500000
Line 13000000
Line 13500000
Line 14000000
Line 14500000
Line 15000000
Line 15500000
Line 16000000
Line 16500000
Line 17000000
Line 17500000
Line 18000000
Line 18500000


18111905

In [4]:
spo = defaultdict(lambda:defaultdict(lambda:set()))
sop = defaultdict(lambda:defaultdict(lambda:set()))
ops = defaultdict(lambda:defaultdict(lambda:set()))
for s, p, o in graph:
    spo[s][p].add(o)
    sop[s][o].add(p)
    ops[o][p].add(s)

In [5]:
Inverse = namedtuple('Inverse', ['base'])

class SparseIdx:
    def __init__(self, graph, n_so, n_p):
        tmp = [scipy.sparse.dok_matrix((n_so, n_so)) for _ in range(n_p)]
        for s, p, o in graph:
            tmp[p][s, o] = 1
        self.pso_bin = [m.tocsr() for m in tmp]
        self.pos_bin = [m.transpose().tocsr() for m in tmp]
        
    def _query(self, *paths):
        def helper(p):
            if isinstance(p, Inverse):
                if isinstance(p.base, Inverse):
                    return helper(p.base.base)
                else:
                    return self.pos_bin[p.base]
            else:
                return self.pso_bin[p]
        result = None
        for path in paths:
            partial = helper(path[0])
            for p in path[1:]:
                partial @= helper(p)
            if result is None:
                result = partial
            else:
                result = result.multiply(partial)
        return result
    
    def count(self, *paths):
        return self._query(*paths).getnnz()
    
    def query(self, *paths):
        tmp = self._query(*paths)
        return [(s, o) for s, o in zip(*tmp.nonzero())]
    
    def pos_neg_query(self, pos_paths, neg_paths):    
        pos = self._query(*pos_paths)
        neg = self._query(*neg_paths)
        tmp = pos - neg.multiply(pos)
        return [(s, o) for s, o in zip(*tmp.nonzero())]

In [6]:
pidx = SparseIdx(graph, len(soenc), len(penc))

In [7]:
def triples_on_path(s, path, o):
    current = s
    p = path[0]
    if len(path) == 1:
        if o in spo[s][p]:
            return [[(s, p, o)]]
    else:
        result = []
        for t in spo[s][p]:
            result += [[(s, p, t)] + triples for triples in triples_on_path(t, path[1:], o) if len(triples) > 0]            
        return result
    return []

------------

In [8]:
so_labels = [None]*len(soenc)
p_labels = [None]*len(penc)

with gzip.open('DBpedia201610/labels_en.ttl.gz', 'rt') as f:
    for n, line in enumerate(f):
        line = line.strip()
        if line[0] == '#' or len(line) == 0:
            continue
        if (n+1)%500000 == 0:
            print("Line", n+1)
        line = line.strip()
        line = line[:-1]
        line = line.split(maxsplit=2)        
        assert len(line) == 3
        assert line[1] == '<http://www.w3.org/2000/01/rdf-schema#label>'
        try:
            s = soenc(line[0])        
        except KeyError:
            continue        
        assert line[2][0] == '"'
        e = line[2].rindex('"')
        l = line[2][1:e]        
        assert so_labels[s] is None or so_labels[s] == l, (s, so_labels[s], line)
        so_labels[s] = l

Line 500000
Line 1000000
Line 1500000
Line 2000000
Line 2500000
Line 3000000
Line 3500000
Line 4000000
Line 4500000
Line 5000000
Line 5500000
Line 6000000
Line 6500000
Line 7000000
Line 7500000
Line 8000000
Line 8500000
Line 9000000
Line 9500000
Line 10000000
Line 10500000
Line 11000000
Line 11500000
Line 12000000
Line 12500000


In [9]:
import rdflib
ontology = rdflib.Graph()
ontology.parse('DBpedia201610/dbpedia_2016-10.owl', format='xml')

for p in range(len(penc)):
    uri = URIRef(penc[p][1:-1])
    labels = ontology.preferredLabel(uri, lang='en')
    assert len(labels) >= 1
    label = str(labels[0][1])
    p_labels[p] = label

In [10]:
def shorten(struct):
    if isinstance(struct, str):
        if struct[0] == '<' and struct[-1] == '>':
            prefixes = [('dbo:', 'http://dbpedia.org/ontology/'), ('dbr:', 'http://dbpedia.org/resource/')]
            struct = struct[1:-1]
            for p, ns in prefixes:
                if struct.startswith(ns):
                    return p + struct[len(ns):]
        return struct
    else:
        return [shorten(item) for item in struct]

-----------------
Idea: wyszukujemy częste wzorce przechodząc od grupy obiektów do grupy obiektów, w sumie podobnie jak SLDM.
Obiekty muszą być ważone, tak jak to było w SLDM, żeby poradzić sobie z "zapadnieciem sie sciezki", np. (si, o) i=1...100

Próby jednoczesnego liczenia odwrotności są zbyt kosztowne, *ale*: odwrotność properties to transpozycja macierzy, a rozpoznawanie sprowadzi się wtedy do zliczania NNZ w point-wise iloczynie.

In [11]:
def find_frequent_patterns(S, O, min_sup, max_len, accept, reject, current_path = []):        
    if accept(current_path):
        args = [S[s] for s in S.keys() & O]
        if len(args) >= 1:
            sup_set = set.union(*args)
            sup = len(sup_set)
            if sup >= min_sup:
                return [(current_path, sup_set)]
    if reject(current_path):
        return []
    if max_len == 0:
        return []
    ctr = defaultdict(lambda:defaultdict(lambda: set()))
    for s, sup_set in S.items():
        for p, objs in spo[s].items():
            for o in objs:
                ctr[p][o] |= sup_set        
#         for p, subjs in ops[s].items():
#             for o in subjs:
#                 ctr[Inverse(p)][o] |= sup_set
    result = []
    for p, objs in ctr.items():
        sup = len(set.union(*objs.values()))    
        if sup >= min_sup:
            result += find_frequent_patterns(objs, O, min_sup, max_len-1, accept, reject, current_path+[p])
    return result

In [12]:
def is_unwanted(p):
    l = p_labels[p]
    keywords = ['spouse', 'successor', 'predecessor', 'previous', 'following', 'subsequent']
    return any([k in l for k in keywords])

unwanted = {p for p in range(len(penc)) if is_unwanted(p)}

def reject(path):
    return len(path) > 0 and path[-1] in unwanted

In [13]:
def process(r, cfg):
    max_len = cfg['max_len']
    min_sup = cfg['min_sup']
    min_conf = cfg['min_conf']
    r_path = [r]
    
    S = {}
    O = set()
    for s, o in pidx.query(r_path):
        S[s] = {s}
        O.add(o)
        
    accept = lambda path: len(path) >= 1 and (path[0] != r or len(path) >= 2)    
    FP = find_frequent_patterns(S, O, min_sup, max_len, accept, reject)
    
    result = []    
    for path, sup_set in FP:        
        if len(path) > 2 and path[0] == path[-1]:
            continue    
        path_sup = pidx.count(path)
        sup = pidx.count(path, [r])
        counterevidence = pidx.pos_neg_query([path], [r_path])
        conf = sup / path_sup
        #print([shorten(penc[p]) for p in path], sup, conf)
        if conf >= min_conf:
            desc = {          
                'support': sup,
                'confidence': conf,
                'counterevidence': counterevidence
            }
            result.append((path, desc))
    return result

#process(penc('<http://dbpedia.org/ontology/isPartOf>'), {'max_len': 2, 'min_sup': 10000, 'min_conf': .95})

In [14]:
with concurrent.futures.ProcessPoolExecutor(4) as executor:           
    cfg = {'max_len': 2, 'min_sup': 100, 'min_conf': .95}
    properties = list(range(len(penc)))        
    results = executor.map(process, properties, [cfg]*len(penc))
    all_paths = dict(zip(properties, results))

In [15]:
all_paths = {r: {tuple(path): info for path, info in paths} for r, paths in all_paths.items()}

In [16]:
decoded = {}

for r, paths in all_paths.items():
    if len(paths) == 0:
        continue
    decoded_paths = []    
    for path, info in paths.items():
        key = tuple((penc[p] for p in path))
        ce = list([(soenc[s], soenc[o]) for s, o in info['counterevidence']])
        decoded_paths.append((key, {'support': info['support'],
                                                  'confidence': info['confidence'],
                                                  'counterevidence': ce
                                                 }))
    decoded[penc[r]] = decoded_paths        

In [17]:
fn = 'decoded-{}.json.gz'.format(datetime.now())

with gzip.open(fn, 'wt') as f:
    json.dump({'cfg': cfg, 'decoded': decoded}, f)